In [1]:
%pip install exifread folium

Note: you may need to restart the kernel to use updated packages.


In [1]:
# pip install exifread folium

import exifread
import folium

import base64
import mimetypes
from pathlib import Path

In [ ]:
colors = ["red", "blue", "green", "purple", "orange", "yellow"]

TEAM_COLOR = "orange"  # Choose a color for your team from the list above

In [8]:
def _ratio_to_float(r):
    return float(r.num) / float(r.den)

def _dms_to_deg(values, ref):
    d = _ratio_to_float(values[0])
    m = _ratio_to_float(values[1])
    s = _ratio_to_float(values[2])
    deg = d + (m / 60.0) + (s / 3600.0)
    if ref in ("S", "W"):
        deg = -deg
    return deg

def extract_gps_from_photo(photo_path):
    with open(photo_path, "rb") as f:
        tags = exifread.process_file(f, details=False)

    lat_vals = tags.get("GPS GPSLatitude")
    lat_ref  = tags.get("GPS GPSLatitudeRef")
    lon_vals = tags.get("GPS GPSLongitude")
    lon_ref  = tags.get("GPS GPSLongitudeRef")

    if not (lat_vals and lat_ref and lon_vals and lon_ref):
        raise ValueError("No EXIF GPS found in this photo.")

    lat = _dms_to_deg(lat_vals.values, str(lat_ref.values))
    lon = _dms_to_deg(lon_vals.values, str(lon_ref.values))
    return lat, lon



In [9]:
# ...existing code...
def make_map_from_media(media_inputs: list, out_html="photo_map_multiple.html", zoom_start=17, marker_color="purple"):
    """
    Accepts a list of inputs (strings or dicts). Each input can be:
      - a string path to an image file (EXIF GPS will be used)
      - a dict with keys:
          - image: path to image file (optional)
          - audio: path to wav/mp3 file (optional)
          - description: short text (optional)
          - lat, lon: numeric coords (optional - required for audio-only items)
    For each item with valid lat/lon the map will add a marker. Clicking the marker
    opens a small popup box that always shows filename + coords and, when available,
    a thumbnail image, description text and an <audio> player (embedded as data URI).
    """
    from html import escape

    items = []

    def _add_item(p_image, p_audio, desc, lat, lon):
        items.append({"image": p_image, "audio": p_audio, "desc": desc, "lat": lat, "lon": lon})

    for entry in media_inputs:
        # normalize to dict
        if isinstance(entry, (str, Path)):
            p = Path(entry)
            ext = p.suffix.lower()
            if ext in {".jpg", ".jpeg", ".png", ".webp", ".gif", ".tiff"}:
                try:
                    lat, lon = extract_gps_from_photo(str(p))
                    _add_item(p, None, None, lat, lon)
                except Exception as e:
                    print(f"Skipping image without GPS: {p.name}: {e}")
            else:
                print(f"Skipping unsupported file (use dict with lat/lon for audio): {p}")
            continue

        if not isinstance(entry, dict):
            print("Unsupported media input, must be path or dict:", entry)
            continue

        p_image = Path(entry["image"]) if entry.get("image") else None
        p_audio = Path(entry["audio"]) if entry.get("audio") else None
        desc = entry.get("description") or entry.get("desc") or ""
        lat = entry.get("lat")
        lon = entry.get("lon")

        # attempt to extract coords from image if not provided
        if (lat is None or lon is None) and p_image:
            try:
                lat, lon = extract_gps_from_photo(str(p_image))
            except Exception:
                lat = lon = None

        # for audio-only entries require lat/lon
        if p_audio and (lat is None or lon is None):
            print(f"Audio item requires explicit lat/lon, skipping: {p_audio.name}")
            continue

        if lat is None or lon is None:
            print(f"No GPS for item, skipping: image={p_image}, audio={p_audio}")
            continue

        _add_item(p_image, p_audio, desc, float(lat), float(lon))

    if not items:
        raise ValueError("No media items with valid coordinates found.")

    # center map
    avg_lat = sum(it["lat"] for it in items) / len(items)
    avg_lon = sum(it["lon"] for it in items) / len(items)
    m = folium.Map(location=[avg_lat, avg_lon], zoom_start=zoom_start, tiles="OpenStreetMap")

    for it in items:
        p_image = it["image"]
        p_audio = it["audio"]
        desc = it["desc"] or ""
        lat = it["lat"]
        lon = it["lon"]

        parts = []
        # image thumbnail
        if p_image and p_image.exists():
            try:
                mime, _ = mimetypes.guess_type(str(p_image))
                mime = mime or "image/png"
                b64 = base64.b64encode(p_image.read_bytes()).decode("ascii")
                parts.append(
                    f'<div style="text-align:center;">'
                    f'<img src="data:{mime};base64,{b64}" '
                    f'style="max-width:280px; max-height:180px; width:auto; height:auto; display:block; margin:0 auto; border-radius:4px;" />'
                    f'</div>'
                )
            except Exception as e:
                parts.append(f'<div><em>Failed to embed image: {escape(str(p_image))}</em></div>')

        # description
        if desc:
            parts.append(f'<div style="font-size:12px; margin-top:6px; color:#222;">{escape(desc)}</div>')

        # audio player
        if p_audio and p_audio.exists():
            try:
                amime, _ = mimetypes.guess_type(str(p_audio))
                amime = amime or "audio/wav"
                a64 = base64.b64encode(p_audio.read_bytes()).decode("ascii")
                parts.append(
                    f'<div style="margin-top:6px; text-align:center;">'
                    f'<audio controls style="width:90%;">'
                    f'<source src="data:{amime};base64,{a64}" type="{amime}"/>'
                    "Your browser does not support the audio element."
                    f'</audio></div>'
                )
            except Exception as e:
                parts.append(f'<div><em>Failed to embed audio: {escape(str(p_audio))}</em></div>')

        # filename + coords (always present)
        filename_line = ""
        if p_image:
            filename_line = escape(p_image.name)
        elif p_audio:
            filename_line = escape(p_audio.name)
        parts.append(f'<div style="font-size:11px; margin-top:6px; color:#444; text-align:center;">{filename_line}<br/><span style="font-family:monospace;">{lat:.6f}, {lon:.6f}</span></div>')

        html = "<div>" + "".join(parts) + "</div>"
        # choose iframe size depending on presence of audio
        iframe_height = 260 if p_audio else 200
        iframe = folium.IFrame(html, width=340, height=iframe_height)
        popup = folium.Popup(iframe, max_width=360)
        folium.Marker([lat, lon], 
                      popup=popup, tooltip=filename_line or "media",
                      icon=folium.Icon(color=marker_color)
                      ).add_to(m)

    # show lat/lon when user clicks on the map
    m.add_child(folium.LatLngPopup())

    try:
        from folium.plugins import MousePosition
        m.add_child(MousePosition(position="topright", separator=" , "))
    except Exception:
        pass

    m.save(out_html)
    return out_html
# ...existing code...

In [10]:
media=[
        {"image":"./data/IMG_1962.jpg","description":"Nice mural","audio":"./data/test_01.mp3"},
        {"image":"./data/IMG_1964.jpg","description":"Nice mural","audio":"./data/test_01.mp3"},
        {"image":"./data/IMG_1967.jpg","description":"Busy road infront of the arquitecture department, smell of trees","audio":"./data/test_02.mp3"}
        
    ]

make_map_from_media(media, marker_color=TEAM_COLOR)

'photo_map_multiple.html'

'map_with_glb_http.html'